<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Creating the Golden Record and Possible Uses
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>


<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Companies rely on accurate customer valuations to correctly determine how to treat certain classes, what offers to serve them, and how much effort to exert in retaining them. Multi-channel companies often have silos of data from separate sales channels and supplement the data using third-party sources. These multi-channel companies need to combine the data souces to create a holistic view of their customers, particularly their valuation. This demonstration showcases Vantage's capabilities in matching customers from different datasets.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Data analysts match customers across different data sources by calculating the similarity between two names (or other fields) using one of several distance measurements. Steps involved in matching two strings are:</p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '50%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Select one string to be the base and the other to be the target</li>
                <br>
                <li>Select a string comparison type to use. Examples include Jaro, Jaro-Winkler, Hamming, and cosine</li>
                <br>
                <li>Determine the number of insertions, deletions, replacements, and tranpositions needed to morph the base string into the target string</li>
                <br>
                <li>Calculate the similarity of the two strings by using the formula for the string comparison type chosen and the insertions, deletions, replacements, and transposes</li>
            </ol>
        </td>
        <td><img src = 'images/counting.png' width = '250'></td>
    </tr>
</table>


<b style = 'font-size:16px;font-family:Arial;color:#00233C'>Leverage native Vantage processing for efficient data field matching workflows</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>ClearScape Analytics provides a StringSimilarity function to calculate how close two key strings are. Typically, analysts will use different key strings depending on the lists. For example, matching on name works when combining two customer lists, whereas matching on physical addresses works when comparing a customer list with a household list.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Data engineers  combine lists two at a time. For instance, combining four lists A, B, C, and D involves three matching steps. The first step combines A and B to form AB. The second step combines lists AB with C to form ABC. The final step combines ABC with D to form ABCD. Each step requires its own matching table and possibly different key strings.</p>

<img src = 'images/fourlists.png' width = 50% align = "center">


<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Data</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The data for this demonstration is based on a brick-and-mortar equipment distributor selling to farmers in four states: New York, Ohio, Pennsylvania, and West Virginia. Over time, this company built a reliable customer list. As customers asked for electronic monitoring service, the company employed an online partner to sell this product. The company also outsourced its call center to a third-party to handle customer service at a cost. Finally, the company worked with a digital marketing group to place cookies on website visitors.</p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The goal is to create a golden record of customers by combining the four lists mentioned above.  Steps in this demonstration are as follows:</p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '50%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Analyze the raw data from each of the four sources</li>
                <br>
                <li>Append matched online data to the equipment data</li>
                <br>
                <li>Append call center data</li>
                <br>
                <li>Append tagging data</li>
                <br>
                <li>Calculate lifetime value</li>
            </ol>
        </td>
        <td><img src = 'images/process.png' width = '250'></td>
    </tr>
</table>

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>1. Connect to Vantage </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [1]:
%connect local, hidewarnings=true

Password: ········


Success: 'local' connection established and activated for user 'demo_user', with default database 'demo_user'


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>


In [2]:
Set query_band='DEMO=CustomerIdentityMatching_SQL.ipynb;' update for session;

Success: 1 rows affected

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string.</p>


In [3]:
--call get_data('DEMO_Customer360_cloud'); ---takes about 10 seconds
call get_data('DEMO_Customer360_local'); ---takes about 30 seconds

Success: 0 rows affected

Success: 0 rows affected

,Message
1,That ran for 0:00:35.07 with 25 statements and 0 errors.


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – if you want to see status of databases/tables created and space used.</p>


In [4]:
call space_report();  -- optional, takes about 10 seconds

Success: 0 rows affected

Success: 0 rows affected

,Space_Report
1,"You have: #databases=4 #tables=53 #views=6 You have used 14.7 MB of 27,890.4 MB available - 0.1% ... Space Usage OK"
2,
3,Database Name #tables #views Avail MB Used MB
4,"demo_user 48 1 27,822.3 MB 14.1 MB"
5,DEMO_Customer360 0 4 0.0 MB 0.0 MB
6,DEMO_Customer360_db 4 0 58.0 MB 0.4 MB
7,DEMO_GraphAnalysis 0 1 0.0 MB 0.0 MB
8,DEMO_GraphAnalysis_db 1 0 10.1 MB 0.2 MB


<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Analyze Raw Data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will inspect each of the four data sets, and determine what fields to use from each table to create a comparison key string.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will inspect:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>The equipment distributor's customer table</li>
    <li>The electronic monitor service's e-commerce list relative to the customer table</li>
    <li>The call center list relative to the customer table</li>
    <li>The tagging list relative to the customer table</li>        
</ol>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Replace the following with a valid connection name:</p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.1- Inspect the Equipment Reseller's Customer Table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The simple SQL select query below shows that the fields include a customer ID, gender, name, address, and a lifetime value for this channel.</p>

In [5]:
SELECT TOP 5 * FROM DEMO_Customer360.Equipment

,CUST_ID,GENDER,FIRSTNAME,LASTNAME,ADDRESS,STATE,CITY,ZIP,EQUIP_LTV
1,19579,M,Pedro,Gonsalves,1233 El Camino Real,NY,Greenville,10586,6159
2,19688,F,Luiza,Teixeira,1522 North U.S. Highway 1,OH,Kerr,45643,5894
3,22143,M,Lucas,Moreau,1455 Sixth Street,OH,Winchester,45697,6299
4,22142,F,Beatriz,Gonsalves,1212 Fifth Street,OH,Fairfield,45018,6463
5,20575,F,Laura,DeSilva,11 Red Mill Road,PA,New Castle,16105,5652


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.2- Inspect E-Commerce Table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A similar SQL query shows this table has a loyalty number, sex, name, physical address, email address and e-commerce lifetime value. Since both tables' records are at an individual level, we will use first name, last name, and city as a matching key string. Fields to import for now include the loyalty number, and email address.</p>

In [6]:
SELECT TOP 5 * FROM DEMO_Customer360.Online

,LOYALTY_NUM,SEX,FNAME,LNAME,ADDR,STATE,CITY,ZIP,EMAIL,ONLINE_LTV
1,5969239,M,Pedro,Gonsalves,1233 El Camino Real,NY,Greenville,10586,pedro.gonsalves@teratitan.com,465
2,6428173,F,Luiza,Teixeira,1522 North U.S. Highway 1,OH,Kerr,45643,lteixeira@gmail.com,936
3,1481403,M,Lucas,Moreau,1455 Sixth Street,OH,Winchester,45697,lucas.moreau@teratitan.com,497
4,1584600,F,Beatriz,Gonsalves,1212 Fifth Street,OH,Fairfield,45018,beatriz.gonsalves@teratitan.com,1708
5,1233911,F,Laura,DeSilva,11 Red Mill Road,PA,New Castle,16105,ldesilva@gmail.com,1766


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.3- Inspect Call Center List</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Note that this table also has records at an individual level, so we can use the same fields to create a matching key. Unique fields to this table include the phone number and call center cost. After matching, we will import the phone number.</p>

In [7]:
SELECT TOP 5 * FROM DEMO_Customer360.CallCenter

,GENDER,FIRST_NAME,LAST_NAME,ADDRESS,STATE,CITY,ZIPCODE,PHONE,CALL_CENTER_COST
1,M,Pedro,Gonsalves,1233 El Camino Real,NY,Greenville,10586,(434) 432-6306,75
2,F,Luiza,Teixeira,1522 North U.S. Highway 1,OH,Kerr,45643,(253) 470-1804,75
3,M,Lucas,Moreau,1455 Sixth Street,OH,Winchester,45697,(941) 929-3922,25
4,F,Beatriz,Gonsalves,1212 Fifth Street,OH,Fairfield,45018,(510) 563-2532,50
5,F,Laura,DeSilva,11 Red Mill Road,PA,New Castle,16105,(570) 387-2031,25


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.4 Inspect Cookie List</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Since cookies are collected at the household level and there are no names provided, we will use address as the matching key string. We will append the cookie and campaign to any matching records.</p>

In [8]:
SELECT TOP 5 * FROM DEMO_Customer360.Cookie

,ADDR_LN_1_TXT,ST_OR_PROV_CD,CITY_NAME,ZIP,COOKIE,CAMPAIGN
1,1233 El Camino Real,NY,Greenville,10586,9AB4BD59BCF976409A5887ED77EB333,Social
2,1522 North U.S. Highway 1,OH,Kerr,45643,F7C0C15D6A14F6206CAABF7B6E45477,Search
3,1455 Sixth Street,OH,Winchester,45697,B0273ABF9237CD15EA547F97104D46FA,Email
4,1212 Fifth Street,OH,Fairfield,45018,44C13D702EBA675AE341AC9BF9C1B89,Email
5,11 Red Mill Road,PA,New Castle,16105,AB7DBCFFD0AD5577560D8B545A842B41,Display


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.5 Inspect Customer Across Lists</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will also examine one customer closely to see how his information is matched throughout the lists. This customer is Jessica Rego, and she (or her address) appears in all four tables.  Note that her name is slightly misspelled in the e-commerce (as Jessika), her city is misspelled in call center table (as Nashporte), and her address is misspelled (with no space between 3114 and Park) in the cookie table. </p>

In [9]:
-- Find Jessica Rego in in-store table
SELECT * FROM DEMO_Customer360.Equipment
WHERE FIRSTNAME = 'Jessica' AND LASTNAME = 'Rego'

,CUST_ID,GENDER,FIRSTNAME,LASTNAME,ADDRESS,STATE,CITY,ZIP,EQUIP_LTV
1,21903,F,Jessica,Rego,3114 Park Street,OH,Nashport,43830,6952


In [10]:
-- Find Jessica Rego in online table
SELECT * FROM DEMO_Customer360.Online
WHERE FNAME LIKE 'Jes%' AND LNAME LIKE 'Reg%'

,LOYALTY_NUM,SEX,FNAME,LNAME,ADDR,STATE,CITY,ZIP,EMAIL,ONLINE_LTV
1,1460445,F,Jessika,Rego,3114 Park Street,OH,Nashport,43830,jrego@yahoo.com,1696


In [11]:
-- Find Jessica Rego in call center table
SELECT * FROM DEMO_Customer360.CallCenter
WHERE FIRST_NAME LIKE 'Jes%' AND LAST_NAME LIKE 'Reg%'

,GENDER,FIRST_NAME,LAST_NAME,ADDRESS,STATE,CITY,ZIPCODE,PHONE,CALL_CENTER_COST
1,F,Jessica,Rego,3114 Park Street,OH,Nashporte,43830,(530) 696-4308,50


In [12]:
-- Find Jessica Rego's address in tagging table
SELECT * FROM DEMO_Customer360.Cookie
WHERE ADDR_LN_1_TXT LIKE '3114%Park%'

,ADDR_LN_1_TXT,ST_OR_PROV_CD,CITY_NAME,ZIP,COOKIE,CAMPAIGN
1,3114Park Street,OH,Nashport,43830,2846E54D5FFE2905099AC6E23014DF7,Email


<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4. Append Online Data to Equipment Table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This section will illustrate matching between the equipment and e-commerce tables using the StringSimilarity function built into ClearScape Analytics. The <a href="https://docs.teradata.com/r/Database-Analytic-Functions/July-2021/Advanced-SQL-Engine-Analytic-Functions/StringSimilarity-SQL-Engine">StringSimiliarity documentation</a> contains more information on the function. The StringSimiliarty function supports 10 different ways of comparing two strings. We will use the Jaro similiarity, which accounts for the number of matching characters and transpositions.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Steps for this section include: 
    <img src = 'images/step4.png' width = '350' align='right' padding='padding:50px'></p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
        <li>Exploring which key strings will be matched from a full join</li> 
        <li>Creating a matching table consisting of: 
        <ul><li>The key string from the equipment table</li>
            <li>The key string from the e-commerce table</li>
            <li>The matching score</li>
            <li>Any columns from the e-commerce table we want to append</li>
        </ul></li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Joining the in-store table with the matching table to form an intermediate golden record</li>
    </ol>
    

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.1 - Exploring full join key strings</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will create a key string for each row of the equipment and online tables. The key string will consist of the first name, last name, and city with all spaces removed. Then each key string from the retail table will be compared with each key string in the e-commerce table. The statement below shows the first 20 rows of the comparison. Note that every equipment key is being compared to the first online key.</p>

In [13]:
SELECT TOP 5
    e.CUST_ID,
    o.LOYALTY_NUM,
    REGEXP_REPLACE(OREPLACE((e.GENDER||e.FIRSTNAME||e.LASTNAME||e.CITY),
                            ' ',''), 
                   '[^a-zA-Z\d:]','',1,0) as EQUIPMENT_KEY,
    REGEXP_REPLACE(OREPLACE((o.SEX||o.FNAME||o.LNAME||o.CITY),
                            ' ',''), 
                   '[^a-zA-Z\d:]','',1,0) as ONLINE_KEY
FROM DEMO_Customer360.Equipment e, DEMO_Customer360.Online o;

,CUST_ID,LOYALTY_NUM,EQUIPMENT_KEY,ONLINE_KEY
1,19579,1233911,MPedroGonsalvesGreenville,FLauraDeSilvaNewCastle
2,19688,1233911,FLuizaTeixeiraKerr,FLauraDeSilvaNewCastle
3,22143,1233911,MLucasMoreauWinchester,FLauraDeSilvaNewCastle
4,22142,1233911,FBeatrizGonsalvesFairfield,FLauraDeSilvaNewCastle
5,20575,1233911,FLauraDeSilvaNewCastle,FLauraDeSilvaNewCastle


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.2 - Creating a matching table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will run the StringSimiliarity function on each row of the table above and only select the rows with a Jaro match of at least a certain threshold. Since the Jaro simliarity score is normalized between 0 and 1, we will choose a threshold of 0.90.</p>

In [14]:
REPLACE VIEW C360_MatchAB_V AS (
    SELECT * FROM StringSimilarity (
        ON (
            SELECT 
                e.CUST_ID, 
                o.LOYALTY_NUM, 
                CAST(REGEXP_REPLACE(OREPLACE((e.GENDER||e.FIRSTNAME||e.LASTNAME||e.CITY),' ',''), 
                                             '[^a-zA-Z\d:]','',1,0) as VARCHAR(50)) 
                                             as EQUIPMENT_KEY,
                CAST(REGEXP_REPLACE(OREPLACE((o.SEX||o.FNAME||o.LNAME||o.CITY),' ',''), 
                                             '[^a-zA-Z\d:]','',1,0) as VARCHAR(50)) 
                                             as ONLINE_KEY,
                o.email
            FROM DEMO_Customer360.Equipment e, DEMO_Customer360.Online o
        ) PARTITION BY ANY
        USING
        ComparisonColumnPairs(
            'jaro (EQUIPMENT_KEY, ONLINE_KEY) AS jaro'
        )
        CaseSensitive('false')
        Accumulate('CUST_ID', 'LOYALTY_NUM', 'EQUIPMENT_KEY', 'ONLINE_KEY', 'EMAIL')
    ) AS dt 
WHERE jaro > .90)

Success: 0 rows affected

In [15]:
SELECT Top 20 * FROM C360_MatchAB_V ORDER BY jaro DESC;

,CUST_ID,LOYALTY_NUM,EQUIPMENT_KEY,ONLINE_KEY,EMAIL,jaro
1,20521,6947731,FBrunaGonsalvesHighlandFalls,FBrunaGonsalvesHighlandFalls,bgonsalves@gmail.com,1
2,19397,5353875,FBeatrizMartinsAshley,FBeatrizMartinsAshley,bmartins@gmail.com,1
3,22142,1584600,FBeatrizGonsalvesFairfield,FBeatrizGonsalvesFairfield,beatriz.gonsalves@teratitan.com,1
4,19463,7782697,FAaryaPatelWaiteville,FAaryaPatelWaiteville,apatel@gmail.com,1
5,19939,5698076,MJuanTorresBretz,MJuanTorresBretz,jtorres@gmail.com,1
6,19579,5969239,MPedroGonsalvesGreenville,MPedroGonsalvesGreenville,pedro.gonsalves@teratitan.com,1
7,21832,1596672,FJiaojieGaoLakeAriel,FJiaojieGaoLakeAriel,jgao@gmail.com,1
8,22143,1481403,MLucasMoreauWinchester,MLucasMoreauWinchester,lucas.moreau@teratitan.com,1
9,21964,1937953,MStevenSugarHamilton,MStevenSugarHamilton,Steven.Sugar@teratitan.com,1
10,19688,6428173,FLuizaTeixeiraKerr,FLuizaTeixeiraKerr,lteixeira@gmail.com,1


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that the first 14 rows consist of exact matches. But starting with the 15th row, the Jaro similarity score is less than 1, indicating there are some discrepancies. For example, taking a look at Jessica Rego, we see that her first name is spelled as Jessika in the online key field:</p>

In [16]:
SELECT * FROM C360_MatchAB_V WHERE EQUIPMENT_KEY LIKE '%JessicaRego%'

,CUST_ID,LOYALTY_NUM,EQUIPMENT_KEY,ONLINE_KEY,EMAIL,jaro
1,21903,1460445,FJessicaRegoNashport,FJessikaRegoNashport,jrego@yahoo.com,0.9666666666666667


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We find that out of the 100 records in the retail table and 70 in the e-commerce table, 67 were matched.</p>

In [17]:
SELECT COUNT(*) FROM DEMO_Customer360.Equipment

,Count(*)
1,100


In [18]:
SELECT COUNT(*) FROM DEMO_Customer360.Online

,Count(*)
1,70


In [19]:
SELECT COUNT(*) FROM C360_MatchAB_V

,Count(*)
1,67


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.3 - Join tables to create intermediate golden record</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Using the CUST_ID and LOYALTY_NUM fields from the above match table, we create an interim golden record from merged retail and e-commerce tables.</p>

In [20]:
REPLACE VIEW C360_gr1_V AS (
    SELECT e.CUST_ID, mab.LOYALTY_NUM, e.GENDER, 
           e.FIRSTNAME, e.LASTNAME, 
           e.ADDRESS, e.STATE, e.CITY, e.ZIP, 
           mab.EMAIL 
    FROM DEMO_Customer360.Equipment e LEFT JOIN C360_MatchAB_V mab
    ON e.CUST_ID = mab.CUST_ID
)

Success: 0 rows affected

In [21]:
SELECT Top 10 * FROM C360_gr1_V;

,CUST_ID,LOYALTY_NUM,GENDER,FIRSTNAME,LASTNAME,ADDRESS,STATE,CITY,ZIP,EMAIL
1,22142,1584600,F,Beatriz,Gonsalves,1212 Fifth Street,OH,Fairfield,45018,beatriz.gonsalves@teratitan.com
2,20575,1233911,F,Laura,DeSilva,11 Red Mill Road,PA,New Castle,16105,ldesilva@gmail.com
3,20003,,M,Ethan,Martin,7693 Cedar Street,PA,Greensboro,15338,
4,20350,1143641,M,Clement,Vincent,6236 Burnt Mill Road,OH,Edon,43518,clement.vincent@teratitan.com
5,19980,,F,Maria,Silva,8147 Center Mill Road,WV,Kellysville,24732,
6,19921,5729451,M,Thiago,Pena,2591 Seventh Street,PA,Millerstown,17062,tpena@gmail.com
7,19513,,M,Cheung,Yang,7462 Hollywood Boulevard,PA,Richland,17087,
8,19553,5451688,M,Bo,Li,3248 N. Torrey Pines Road,PA,Oak Ridge,16245,bo.li@teratitan.com
9,19614,,M,Clement,Bertrand,5606 Paper Mill Road,NY,Rochester,14660,
10,19372,1118440,F,Fei,Zhou,5957 Hill Street,PA,Harrisburg,17128,fzhou@outlook.com


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see Jessica's record has been updated with an email address.</p>

In [22]:
SELECT * FROM C360_gr1_V WHERE FIRSTNAME = 'Jessica' AND LASTNAME = 'Rego'

,CUST_ID,LOYALTY_NUM,GENDER,FIRSTNAME,LASTNAME,ADDRESS,STATE,CITY,ZIP,EMAIL
1,21903,1460445,F,Jessica,Rego,3114 Park Street,OH,Nashport,43830,jrego@yahoo.com


<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>5. Append Call Center Data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We repeat the process, this time appending the call center data.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Steps for this section include:
    <img src = 'images/step5.png' width = '300' align='right' padding='padding:50px'></p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Creating a matching table consisting of:
        <ul><li>The key string from the intermediate golden record table</li>
            <li>The key string from the call center table</li>
            <li>The matching score</li>
            <li>Any columns from the call center table we want to append</li>
        </ul></li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Joining the intermediate golden record table with the matching table to form a new intermediate golden record</li>
    </ol>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1 - Creating a matching table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>As in the previous section, we will create a view consisting of rows with a Jaro match greater than 0.9.</p>

In [23]:
REPLACE VIEW C360_MatchABC_V AS (
    SELECT * FROM StringSimilarity (
        ON (
            SELECT 
                gr1.CUST_ID, 
                CAST(REGEXP_REPLACE(OREPLACE((gr1.FIRSTNAME||gr1.LASTNAME||gr1.CITY),' ',''), 
                                             '[^a-zA-Z\d:]','',1,0) as VARCHAR(50)) 
                                             as GR1_KEY,
                CAST(REGEXP_REPLACE(OREPLACE((cc.FIRST_NAME||cc.LAST_NAME||cc.CITY),' ',''), 
                                             '[^a-zA-Z\d:]','',1,0) as VARCHAR(50)) 
                                             as CALLCENTER_KEY,
                cc.PHONE                               
            FROM C360_gr1_V gr1, DEMO_Customer360.CallCenter cc
        ) PARTITION BY ANY
        USING
        ComparisonColumnPairs(
            'jaro (GR1_KEY, CALLCENTER_KEY) AS jaro'
        )
        CaseSensitive('false')
        Accumulate('CUST_ID', 'GR1_KEY', 'CALLCENTER_KEY', 'PHONE'/*, 'CALL_CENTER_COST'*/)
    ) AS dt 
WHERE jaro > .90)

Success: 0 rows affected

In [24]:
SELECT * FROM C360_MatchABC_V ORDER BY jaro DESC

,CUST_ID,GR1_KEY,CALLCENTER_KEY,PHONE,jaro
1,21964,StevenSugarHamilton,StevenSugarHamilton,(513) 273-2352,1
2,19939,JuanTorresBretz,JuanTorresBretz,(775) 202-5303,1
3,21604,SofieLarsenJavaCenter,SofieLarsenJavaCenter,(772) 503-2101,1
4,20575,LauraDeSilvaNewCastle,LauraDeSilvaNewCastle,(570) 387-2031,1
5,21832,JiaojieGaoLakeAriel,JiaojieGaoLakeAriel,(504) 787-0477,1
6,19499,MiguelSousaLinden,MiguelSousaLinden,(304) 231-8631,1
7,19553,BoLiOakRidge,BoLiOakRidge,(617) 284-2673,1
8,19688,LuizaTeixeiraKerr,LuizaTeixeiraKerr,(253) 470-1804,1
9,22142,BeatrizGonsalvesFairfield,BeatrizGonsalvesFairfield,(510) 563-2532,1
10,19579,PedroGonsalvesGreenville,PedroGonsalvesGreenville,(434) 432-6306,1


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We find that out of the 30 in the call center table, 27 were matched.</p>

In [25]:
SELECT COUNT(*) FROM DEMO_Customer360.CALLCENTER

,Count(*)
1,30


In [26]:
SELECT COUNT(*) FROM C360_MatchABC_V

,Count(*)
1,27


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that Jessica Nashport was matched between the two lists even though her last name was misspelled in the Call Center list. The matching score was 0.983.</p>

In [27]:
SELECT * FROM C360_MatchABC_V WHERE GR1_KEY LIKE '%JessicaRego%'

,CUST_ID,GR1_KEY,CALLCENTER_KEY,PHONE,jaro
1,21903,JessicaRegoNashport,JessicaRegoNashporte,(530) 696-4308,0.9833333333333334


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2 - Join tables to create intermediate golden record</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Using the CUST_ID, we create another interim golden record including data from the call center table.</p>

In [28]:
REPLACE VIEW C360_GR2_V AS (
    SELECT gr.CUST_ID, gr.LOYALTY_NUM, gr.GENDER, 
           gr.FIRSTNAME, gr.LASTNAME, 
           gr.ADDRESS, gr.STATE, gr.CITY, gr.ZIP, 
           --gr.EQUIP_LTV, gr.ONLINE_LTV, 
           --COALESCE(mabc.CALL_CENTER_COST, 0) as CALL_CENTER_COST, 
            gr.EMAIL,
            mabc.PHONE
    FROM C360_GR1_V gr LEFT JOIN C360_MatchABC_V mabc
    ON gr.CUST_ID = mabc.CUST_ID
)

Success: 0 rows affected

In [29]:
SELECT TOP 10 * FROM C360_GR2_V;

,CUST_ID,LOYALTY_NUM,GENDER,FIRSTNAME,LASTNAME,ADDRESS,STATE,CITY,ZIP,EMAIL,PHONE
1,22142,1584600,F,Beatriz,Gonsalves,1212 Fifth Street,OH,Fairfield,45018,beatriz.gonsalves@teratitan.com,(510) 563-2532
2,19513,,M,Cheung,Yang,7462 Hollywood Boulevard,PA,Richland,17087,,
3,19553,5451688,M,Bo,Li,3248 N. Torrey Pines Road,PA,Oak Ridge,16245,bo.li@teratitan.com,(617) 284-2673
4,19372,1118440,F,Fei,Zhou,5957 Hill Street,PA,Harrisburg,17128,fzhou@outlook.com,
5,20003,,M,Ethan,Martin,7693 Cedar Street,PA,Greensboro,15338,,
6,20350,1143641,M,Clement,Vincent,6236 Burnt Mill Road,OH,Edon,43518,clement.vincent@teratitan.com,
7,19614,,M,Clement,Bertrand,5606 Paper Mill Road,NY,Rochester,14660,,
8,19980,,F,Maria,Silva,8147 Center Mill Road,WV,Kellysville,24732,,
9,20575,1233911,F,Laura,DeSilva,11 Red Mill Road,PA,New Castle,16105,ldesilva@gmail.com,(570) 387-2031
10,19921,5729451,M,Thiago,Pena,2591 Seventh Street,PA,Millerstown,17062,tpena@gmail.com,(702) 821-1527


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see Jessica's record has been updated with an email address (from the last join) and a phone number (from this join).</p>

In [30]:
SELECT * FROM C360_GR2_V WHERE FIRSTNAME = 'Jessica' AND LASTNAME = 'Rego'

,CUST_ID,LOYALTY_NUM,GENDER,FIRSTNAME,LASTNAME,ADDRESS,STATE,CITY,ZIP,EMAIL,PHONE
1,21903,1460445,F,Jessica,Rego,3114 Park Street,OH,Nashport,43830,jrego@yahoo.com,(530) 696-4308


<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>6. Append Cookie Data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We repeat the process, this time appending the cookie data. Since cookies are collected at the household level, we will use address as the matching string.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Steps for this section include:
<img src = 'images/step6.png' width = '300' align='right' padding='padding:50px'></p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Creating a matching table
    <li>Joining the intermediate golden record table with the matching table to form a new intermediate golden record</li>
    </ol>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>6.1 - Creating a matching table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>As in the previous section, we will create a view consisting of rows with a Jaro match greater than 0.9.</p>

In [31]:
REPLACE VIEW C360_MatchABCD_V AS (
    SELECT * FROM StringSimilarity (
        ON (
            SELECT 
                gr2.CUST_ID, 
                gr2.EMAIL,
                gr2.PHONE,
                CAST(REGEXP_REPLACE(OREPLACE((gr2.ADDRESS||gr2.CITY),' ',''), 
                                             '[^a-zA-Z\d:]','',1,0) as VARCHAR(50)) 
                                             as GR2_KEY,
                CAST(REGEXP_REPLACE(OREPLACE((t.ADDR_LN_1_TXT||t.CITY_NAME),' ',''), 
                                             '[^a-zA-Z\d:]','',1,0) as VARCHAR(50)) 
                                             as TAGGING_KEY,
                t.COOKIE,
                t.CAMPAIGN
            FROM C360_GR2_V gr2, DEMO_Customer360.Cookie t
        ) PARTITION BY ANY
        USING
        ComparisonColumnPairs(
            'jaro (GR2_KEY, TAGGING_KEY) AS jaro'
        )
        CaseSensitive('false')
        Accumulate('CUST_ID','EMAIL','PHONE', 'GR2_KEY', 'TAGGING_KEY', 'COOKIE','CAMPAIGN')
    ) AS dt 
WHERE jaro > .90)

Success: 0 rows affected

In [32]:
SELECT * FROM C360_MatchABCD_V ORDER BY jaro DESC

,CUST_ID,EMAIL,PHONE,GR2_KEY,TAGGING_KEY,COOKIE,CAMPAIGN,jaro
1,21647,harish.banerjee@teratitan.com,(216) 258-3212,331CityBlvdWCleveland,331CityBlvdWCleveland,3FDBF0376A814344908925F93F3ED59,Search,1
2,20387,tmedina@yahoo.com,(813) 611-8265,2729AbbotKinneyBlvdSpencer,2729AbbotKinneyBlvdSpencer,402F84D4AD94ECF934252A443F6ED42F,Search,1
3,22018,sahil.sharma@teratitan.com,,3739BarleyMillRoadBrooklyn,3739BarleyMillRoadBrooklyn,DD884D7B1EE46E7B6E1BF8DA9B10988F,Search,1
4,20117,maya.campbell@teratitan.com,(304) 903-1275,2585HayesStParkersburg,2585HayesStParkersburg,73C79D2507929CC9284C702263C942,Email,1
5,19403,gsilvan@yahoo.com,,4132WillieMaysPlazaLimestone,4132WillieMaysPlazaLimestone,F657E711ED332EBF9B60A921143A8341,Display,1
6,19939,jtorres@gmail.com,(775) 202-5303,2193UpperSnuffMillRowBretz,2193UpperSnuffMillRowBretz,82E3A91DF019BDAE7210E63B782F,Search,1
7,21964,Steven.Sugar@teratitan.com,(513) 273-2352,1854OakStreetHamilton,1854OakStreetHamilton,15CDB6C74BD9B2929FB0F41359ECEE42,Social,1
8,21604,sofie.larsen@teratitan.com,(772) 503-2101,1600BeverlyBoulevardJavaCenter,1600BeverlyBoulevardJavaCenter,4CF6A88A0664B34BD89ED714A11DF5F,Search,1
9,21903,jrego@yahoo.com,(530) 696-4308,3114ParkStreetNashport,3114ParkStreetNashport,2846E54D5FFE2905099AC6E23014DF7,Email,1
10,21832,jgao@gmail.com,(504) 787-0477,1964ViewStreetLakeAriel,1964ViewStreetLakeAriel,F4056690A7878DA98044A8B55F3A9DE,Search,1


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We find that out of the 45 in the cookie table, 44 were matched.</p>

In [33]:
SELECT COUNT(*) FROM DEMO_Customer360.Cookie

,Count(*)
1,45


In [34]:
SELECT COUNT(*) FROM C360_MatchABCD_V

,Count(*)
1,44


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that Jessica's physical address was matched between the two lists.</p>

In [35]:
SELECT * FROM C360_MatchABCD_V WHERE GR2_KEY LIKE '%3114Park%'

,CUST_ID,EMAIL,PHONE,GR2_KEY,TAGGING_KEY,COOKIE,CAMPAIGN,jaro
1,21903,jrego@yahoo.com,(530) 696-4308,3114ParkStreetNashport,3114ParkStreetNashport,2846E54D5FFE2905099AC6E23014DF7,Email,1


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>6.2 - Join tables to create intermediate golden record</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Using the CUST_ID, we create another interim golden record from merged intermediate golden record and cookie tables.</p>

In [36]:
REPLACE VIEW C360_GR3_V AS (
    SELECT gr2.CUST_ID, gr2.LOYALTY_NUM, gr2.GENDER, 
           gr2.FIRSTNAME, gr2.LASTNAME, 
           gr2.ADDRESS, gr2.STATE, gr2.CITY, gr2.ZIP, gr2.EMAIL,gr2.PHONE,
           mabcd.COOKIE,
           mabcd.CAMPAIGN 
    FROM C360_GR2_V gr2 LEFT JOIN C360_MatchABCD_V mabcd
    ON gr2.CUST_ID = mabcd.CUST_ID
)

Success: 0 rows affected

In [37]:
SELECT TOP 5 * FROM C360_GR3_V

,CUST_ID,LOYALTY_NUM,GENDER,FIRSTNAME,LASTNAME,ADDRESS,STATE,CITY,ZIP,EMAIL,PHONE,COOKIE,CAMPAIGN
1,20350,1143641,M,Clement,Vincent,6236 Burnt Mill Road,OH,Edon,43518,clement.vincent@teratitan.com,,,
2,19372,1118440,F,Fei,Zhou,5957 Hill Street,PA,Harrisburg,17128,fzhou@outlook.com,,,
3,20575,1233911,F,Laura,DeSilva,11 Red Mill Road,PA,New Castle,16105,ldesilva@gmail.com,(570) 387-2031,AB7DBCFFD0AD5577560D8B545A842B41,Display
4,21635,2242698,M,Alexander,Muller,3649 Main Street,PA,York,17407,alexander.muller@teratitan.com,,9856C1982CB8FD2ABF9AC11CB7A9FD,Display
5,19410,6170383,M,Rodrigo,Gonsalves,7150 Second Street,PA,Refton,17568,rgonsalvez@outlook.com,,,


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that Jessica's record has been updated with a cookie and a campaign.</p>

In [38]:
SELECT * FROM C360_GR3_V WHERE FIRSTNAME = 'Jessica' AND LASTNAME = 'Rego'

,CUST_ID,LOYALTY_NUM,GENDER,FIRSTNAME,LASTNAME,ADDRESS,STATE,CITY,ZIP,EMAIL,PHONE,COOKIE,CAMPAIGN
1,21903,1460445,F,Jessica,Rego,3114 Park Street,OH,Nashport,43830,jrego@yahoo.com,(530) 696-4308,2846E54D5FFE2905099AC6E23014DF7,Email


<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>7. Calculate Lifetime Value Across All Channels</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Finally, we calculate the lifetime value by taking into account the channel lifetime value from equipment and online as well as the cost to service a customer from the call center using the formula:<br>
&ensp; LTV = Equipment Purchases + Online Services Sales - Call Center Cost<br>
    Since this is the final calculation, we will write out our results to a table rather than a view.</p>

In [39]:
CREATE TABLE C360_GoldenRecord AS (
    SELECT gr3.CUST_ID, gr3.LOYALTY_NUM, gr3.GENDER, 
           gr3.FIRSTNAME, gr3.LASTNAME, gr3.EMAIL,gr3.PHONE,
           gr3.ADDRESS, gr3.STATE, gr3.CITY, gr3.ZIP, 
           COALESCE(e.EQUIP_LTV,0) as EQUIPLTV , COALESCE(o.ONLINE_LTV,0) as ONLINELTV, COALESCE(c.CALL_CENTER_COST,0) as CALLCENTER_COST,
            gr3.COOKIE, gr3.CAMPAIGN,
           EQUIPLTV + ONLINELTV - CALLCENTER_COST as LTV
    FROM C360_GR3_V gr3 LEFT JOIN DEMO_Customer360.Equipment e ON gr3.CUST_ID = e.CUST_ID
    LEFT JOIN DEMO_Customer360.Online o ON gr3.EMAIL = o.EMAIL
    LEFT JOIN DEMO_Customer360.CallCenter c ON gr3.PHONE= c.PHONE
    
    )
WITH DATA;

Success: 0 rows affected

In [40]:
SELECT TOP 5 * FROM C360_GoldenRecord

,CUST_ID,LOYALTY_NUM,GENDER,FIRSTNAME,LASTNAME,EMAIL,PHONE,ADDRESS,STATE,CITY,ZIP,EQUIPLTV,ONLINELTV,CALLCENTER_COST,COOKIE,CAMPAIGN,LTV
1,19980,,F,Maria,Silva,,,8147 Center Mill Road,WV,Kellysville,24732,5095,0,0,,,5095
2,19364,,F,Chen-chi,Wang,,,7703 Fifth Street,OH,Fredericksburg,44627,6188,0,0,,,6188
3,22018,4479603,M,Sahil,Sharma,sahil.sharma@teratitan.com,,3739 Barley Mill Road,NY,Brooklyn,11252,6198,1931,0,DD884D7B1EE46E7B6E1BF8DA9B10988F,Search,8129
4,19614,,M,Clement,Bertrand,,,5606 Paper Mill Road,NY,Rochester,14660,5705,0,0,,,5705
5,20350,1143641,M,Clement,Vincent,clement.vincent@teratitan.com,,6236 Burnt Mill Road,OH,Edon,43518,5762,649,0,,,6411


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Focusing in on Jessica, we see her lifetime value across all channels.</p>

In [41]:
SELECT * FROM C360_GoldenRecord WHERE FIRSTNAME = 'Jessica' AND LASTNAME = 'Rego'

,CUST_ID,LOYALTY_NUM,GENDER,FIRSTNAME,LASTNAME,EMAIL,PHONE,ADDRESS,STATE,CITY,ZIP,EQUIPLTV,ONLINELTV,CALLCENTER_COST,COOKIE,CAMPAIGN,LTV
1,21903,1460445,F,Jessica,Rego,jrego@yahoo.com,(530) 696-4308,3114 Park Street,OH,Nashport,43830,6952,1696,50,2846E54D5FFE2905099AC6E23014DF7,Email,8598


<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>8. Conclusion</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Thus by combinig the data from Multiple channels we are able to create a golden record for each customers and calculate the value based on the various channels used. This kind of combined data can be used for various kind of analysis including the one shown in this demo which calculates the life time value for each customers based on the various channels used by the customer and the cost involved.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>9. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Clean up the work tables to prevent error next time.</p>

In [42]:
DROP VIEW C360_MatchAB_V;

Success: 12 rows affected

In [43]:
DROP VIEW C360_gr1_V; 

Success: 16 rows affected

In [44]:
DROP VIEW C360_MatchABC_V;

Success: 11 rows affected

In [45]:
DROP VIEW C360_MatchABCD_V; 

Success: 14 rows affected

In [46]:
DROP VIEW C360_GR3_V; 

Success: 19 rows affected

In [47]:
DROP TABLE C360_GoldenRecord;

Success: 30 rows affected

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>


In [48]:
call remove_data('DEMO_Customer360'); 

Success: 0 rows affected

,Message
1,Removed objects related to DEMO_Customer360. That ran for 0:00:04.61


<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>